In [122]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw
import io

def generate_aruco_marker(id, size=500, dictionary=cv2.aruco.DICT_4X4_250):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[1][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    fig, ax = plt.subplots()
    
    ax.scatter(v[:3000,0],v[:3000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:3000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)
    ax.annotate(f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}", xy=(1.05, 0.1), xycoords='axes fraction', ha='right')
    ax.annotate(f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}", xy=(1.05, 0.05), xycoords='axes fraction', ha='right')

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = 6
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    
    fig, v = plot_object('/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/006_mustard_bottle/tsdf/textured.obj')
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=300)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 - 18
    marker_y_pos = 0
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2
    plot_y_pos = a4_pixel_height - plot_pil_img.height - 520

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))
    
    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:3000,1]) - min(v[:3000,1]))
    intersection_pixel_y = (max(v[:3000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 - 18
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + 1300

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    
    a4_img.save('/home/jose/Downloads/Final_Output.pdf', "PDF", resolution=100.0)
    plt.close()
